# 📊 데이터 통합 프로젝트

> **Welcome과 Quickpoll 데이터를 통합하여 통합 데이터셋을 생성하는 프로세스**

---

## 🎯 프로젝트 개요

| 항목 | 내용 |
|------|------|
| **목적** | Welcome 파일 2개와 Quickpoll 파일 35개를 `mb_sn`을 기준으로 통합 |
| **데이터 소스** | `../data/raw/` 폴더의 Excel 파일들 |
| **통합 기준** | `mb_sn` 컬럼을 사용한 Outer Join |
| **결과** | 36,113명의 패널 데이터 통합 |

## 📋 데이터 처리 특징

### 🔹 **Welcome 파일**
- **헤더 위치**: 첫 번째 행 (header=0)
- **파일 수**: 2개
- **내용**: 기본 패널 정보 (인구통계, 기본 설문)

### 🔹 **Quickpoll 파일**
- **헤더 위치**: 두 번째 행 (header=1)
- **파일 수**: 35개
- **내용**: 설문 응답 데이터 (날짜별 설문)

### 🔹 **통합 과정**
- `'고유번호'` 컬럼을 `'mb_sn'`으로 변경
- Outer Join으로 모든 데이터 보존
- 중복 컬럼 자동 정리

---

## 📁 노트북 구조

1. **라이브러리 설정** - 필요한 패키지 import
2. **파일 경로 설정** - 데이터 파일 경로 및 목록 확인
3. **파일 목록 확인** - glob을 사용하여 파일 목록 확인
4. **Welcome 파일 통합** - 기본 패널 데이터 통합
5. **해독표 추출** - 질문/보기 매핑 테이블 생성
6. **Quickpoll 파일 통합** - 설문 응답 데이터 통합
7. **중복 데이터 처리** - 데이터 품질 개선
8. **최종 결과 저장** - 통합된 데이터셋 저장

## 📚 1. 라이브러리 설정

데이터 처리에 필요한 라이브러리들을 import합니다.

In [9]:
import pandas as pd
import glob  # 파일 경로를 쉽게 다룰 수 있게 해주는 라이브러리
import os    # 운영체제와 상호작용하는 라이브러리
import gc    # 가비지 컬렉션을 위한 라이브러리

## 📁 2. 파일 경로 설정

데이터 파일들이 위치한 경로를 설정하고 파일 목록을 확인합니다.

In [10]:
# 경로 설정 및 파일 목록 불러오기
base_path = '../data/raw/'
welcome_path = os.path.join(base_path, 'Welcome/*.xlsx')
quickpoll_path = os.path.join(base_path, 'Quickpoll/*.xlsx')

welcome_files = glob.glob(welcome_path)
quickpoll_files = glob.glob(quickpoll_path)

print(f"Welcome 파일 경로: {welcome_path}")
print(f"Quickpoll 파일 경로: {quickpoll_path}")
print(f"Welcome 파일 개수: {len(welcome_files)}개")
print(f"Quickpoll 파일 개수: {len(quickpoll_files)}개")

Welcome 파일 경로: ../data/raw/Welcome/*.xlsx
Quickpoll 파일 경로: ../data/raw/Quickpoll/*.xlsx
Welcome 파일 개수: 2개
Quickpoll 파일 개수: 35개


## 📋 3. 파일 목록 확인

glob을 사용하여 각 폴더의 Excel 파일들을 찾고 목록을 확인합니다.

In [11]:
# 파일 목록 확인
print("\n=== Welcome 파일 목록 ===")
for i, file in enumerate(welcome_files, 1):
    print(f"{i}. {os.path.basename(file)}")

print("\n=== Quickpoll 파일 목록 ===")
for i, file in enumerate(quickpoll_files, 1):
    print(f"{i}. {os.path.basename(file)}")


=== Welcome 파일 목록 ===
1. Welcome_2nd.xlsx
2. Welcome_1st.xlsx

=== Quickpoll 파일 목록 ===
1. qpoll_join_250106.xlsx
2. qpoll_join_250604.xlsx
3. qpoll_join_250716.xlsx
4. qpoll_join_250624.xlsx
5. qpoll_join_250326.xlsx
6. qpoll_join_250310.xlsx
7. qpoll_join_250605.xlsx
8. qpoll_join_250107.xlsx
9. qpoll_join_250116.xlsx
10. qpoll_join_250317.xlsx
11. qpoll_join_250710.xlsx
12. qpoll_join_250707.xlsx
13. qpoll_join_250204.xlsx
14. qpoll_join_250619.xlsx
15. qpoll_join_250224.xlsx
16. qpoll_join_250623.xlsx
17. qpoll_join_250620.xlsx
18. qpoll_join_250616.xlsx
19. qpoll_join_250704.xlsx
20. qpoll_join_250206.xlsx
21. qpoll_join_250617.xlsx
22. qpoll_join_250226.xlsx
23. qpoll_join_250123.xlsx
24. qpoll_join_250709.xlsx
25. qpoll_join_250610.xlsx
26. qpoll_join_250702.xlsx
27. qpoll_join_250714.xlsx
28. qpoll_join_250221.xlsx
29. qpoll_join_250626.xlsx
30. qpoll_join_250723.xlsx
31. qpoll_join_250627.xlsx
32. qpoll_join_250328.xlsx
33. qpoll_join_250703.xlsx
34. qpoll_join_250611.xlsx
35.

## 🔗 4. Welcome 파일 통합

Welcome 파일들을 먼저 통합하여 기본 DataFrame을 생성합니다.

In [12]:
# Welcome 파일 통합 (header=1 없이)
# Welcome 파일은 깨끗하다고 가정하고 그대로 읽습니다.
print("=== Welcome 파일 통합 시작 ===")
base_df = pd.read_excel(welcome_files[0])
print(f"기본 파일 로드: {os.path.basename(welcome_files[0])}")
print(f"기본 데이터 크기: {base_df.shape}")

# '고유번호' 컬럼을 'mb_sn'으로 변경
if '고유번호' in base_df.columns:
    base_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
    print("'고유번호' 컬럼을 'mb_sn'으로 변경했습니다.")

# 나머지 Welcome 파일들 통합
for file in welcome_files[1:]:
    print(f"병합 중: {os.path.basename(file)}")
    temp_df = pd.read_excel(file)
    print(f"  - 임시 데이터 크기: {temp_df.shape}")
    
    # '고유번호' 컬럼을 'mb_sn'으로 변경
    if '고유번호' in temp_df.columns:
        temp_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
    
    # Welcome 파일끼리는 outer merge로 모든 패널을 포함시킵니다.
    base_df = pd.merge(base_df, temp_df, on='mb_sn', how='outer')
    print(f"  - 통합 후 크기: {base_df.shape}")

print(f"--- Welcome 파일 통합 완료 (기준 데이터 크기: {base_df.shape}) ---")

=== Welcome 파일 통합 시작 ===
기본 파일 로드: Welcome_2nd.xlsx
기본 데이터 크기: (28744, 22)
병합 중: Welcome_1st.xlsx
  - 임시 데이터 크기: (27221, 5)
  - 통합 후 크기: (35240, 26)
--- Welcome 파일 통합 완료 (기준 데이터 크기: (35240, 26)) ---


## 🔍 5. 해독표 추출

모든 Excel 파일에서 해독표(매핑 테이블) 정보를 추출합니다.

> **해독표란?** 질문과 보기 정보를 담은 매핑 테이블로, 설문 응답 데이터를 해석하는 데 사용됩니다.

In [13]:
# 해독표(매핑 테이블) 추출 및 검증
import pandas as pd
import glob
import os

# --- 경로 설정 및 파일 목록 불러오기 ---
base_path = '../data/raw/'
all_files = glob.glob(os.path.join(base_path, '**/*.xlsx'), recursive=True)

print(f"발견된 Excel 파일: {len(all_files)}개")

# 모든 질문/보기 정보를 담을 빈 리스트 생성
all_mappings = []
successful_files = 0
failed_files = 0

print("\n--- 1단계: 모든 파일의 두 번째 시트에서 '해독표' 정보 추출 시작 ---")

for file in all_files:
    filename = os.path.basename(file)
    print(f"처리 중: {filename}")
    
    try:
        # 시트 목록 확인
        excel_file = pd.ExcelFile(file)
        sheet_names = excel_file.sheet_names
        print(f"  - 시트 목록: {sheet_names}")
        
        # 두 번째 시트가 있는지 확인
        if len(sheet_names) >= 2:
            df_map = pd.read_excel(file, sheet_name=1)
            print(f"  - 시트 크기: {df_map.shape}")
            print(f"  - 컬럼: {list(df_map.columns)}")
            
            # 데이터 검증
            if not df_map.empty:
                # 파일 이름을 'source_file' 컬럼으로 추가
                df_map['source_file'] = filename
                all_mappings.append(df_map)
                successful_files += 1
                print(f"  ✅ 성공")
            else:
                print(f"  ⚠️ 빈 시트")
                failed_files += 1
        else:
            print(f"  ⚠️ 두 번째 시트가 없음 (시트 수: {len(sheet_names)})")
            failed_files += 1
            
    except Exception as e:
        print(f"  ❌ 오류: {e}")
        failed_files += 1

print(f"\n--- 처리 결과 ---")
print(f"성공: {successful_files}개 파일")
print(f"실패: {failed_files}개 파일")

# 리스트에 담긴 모든 데이터프레임을 하나로 합치기
if all_mappings:
    master_mapping_df = pd.concat(all_mappings, ignore_index=True)
    
    print(f"\n--- 마스터 해독표 정보 ---")
    print(f"총 행 수: {len(master_mapping_df)}")
    print(f"총 컬럼 수: {len(master_mapping_df.columns)}")
    print(f"출처 파일 수: {master_mapping_df['source_file'].nunique()}")
    
    # 최종 해독표(매핑 테이블) 저장
    output_path = '../data/processed/00_master_mapping_table.csv'
    master_mapping_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"\n🎉 마스터 해독표(매핑 테이블) 생성 완료!")
    print(f"'{output_path}' 경로에 저장되었습니다.")
    display(master_mapping_df.head())
else:
    print("\n❌ 처리할 수 있는 매핑 시트가 없습니다. 파일들을 확인해주세요.")

발견된 Excel 파일: 37개

--- 1단계: 모든 파일의 두 번째 시트에서 '해독표' 정보 추출 시작 ---
처리 중: qpoll_join_250106.xlsx
  - 시트 목록: ['qpoll_join_250106', 'qpoll_20250814173212']
  - 시트 크기: (1, 22)
  - 컬럼: ['설문제목', '보기1', '보기2', '보기3', '보기4', '보기5', '보기6', '보기7', '보기8', '보기9', '보기10', '총참여자수', '보기1_CNT', '보기2_CNT', '보기3_CNT', '보기4_CNT', '보기5_CNT', '보기6_CNT', '보기7_CNT', '보기8_CNT', '보기9_CNT', '보기10_CNT']
  ✅ 성공
처리 중: qpoll_join_250604.xlsx
  - 시트 목록: ['qpoll_join_250604', 'qpoll_20250804145133']
  - 시트 크기: (1, 16)
  - 컬럼: ['설문제목', '보기1', '보기2', '보기3', '보기4', '보기5', '보기6', '보기7', '총참여자수', '보기1_CNT', '보기2_CNT', '보기3_CNT', '보기4_CNT', '보기5_CNT', '보기6_CNT', '보기7_CNT']
  ✅ 성공
처리 중: qpoll_join_250716.xlsx
  - 시트 목록: ['qpoll_join_250716', 'qpoll_20250804160251']
  - 시트 크기: (1, 20)
  - 컬럼: ['설문제목', '보기1', '보기2', '보기3', '보기4', '보기5', '보기6', '보기7', '보기8', '보기9', '총참여자수', '보기1_CNT', '보기2_CNT', '보기3_CNT', '보기4_CNT', '보기5_CNT', '보기6_CNT', '보기7_CNT', '보기8_CNT', '보기9_CNT']
  ✅ 성공
처리 중: qpoll_join_250624.xlsx
  - 시트 목록: ['qpoll_join

,설문제목,보기1,보기2,보기3,보기4,보기5,보기6,보기7,보기8,보기9,...,보기9_CNT,보기10_CNT,source_file,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,변수명,문항,문항유형
0,여러분은 평소 체력 관리를 위해 어떤 활동을 하고 계신가요? 모두 선택해주세요.,헬스,홈트레이닝,요가/필라테스,달리기/걷기,자전거 타기,"스포츠(축구, 배드민턴 등)",수영,등산,기타,...,212.0,1061.0,qpoll_join_250106.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,여러분은 다가오는 여름철 가장 걱정되는 점이 무엇인가요?,더위와 땀,피부 트러블,냉방병,전기요금 부담,체력 저하,휴가 계획 스트레스,특별히 걱정되는 것이 없다,NaN,NaN,...,NaN,NaN,qpoll_join_250604.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,여러분의 휴대폰 갤러리에 가장 많이 저장되어져 있는 사진은 무엇인가요?,셀카/인물 사진,음식 사진,풍경/여행 사진,반려동물 사진,메모용 캡처/스크린샷,친구/가족과의 단체 사진,SNS/인터넷에서 저장한 이미지,"업무/학업 관련 사진(자료, 필기 등)",기타,...,270.0,NaN,qpoll_join_250716.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,여러분은 최근 가장 지출을 많이 한 곳은 어디입니까?,외식비,배달비,옷/쇼핑,"콘서트, 전시 등 문화생활",기타,NaN,NaN,NaN,NaN,...,NaN,NaN,qpoll_join_250624.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,여러분은 올해 해외여행을 간다면 어디로 가고 싶나요? 모두 선택해주세요,유럽,동남아,미국/캐나다,일본/중국,기타,해외여행을 가고싶지 않다,NaN,NaN,NaN,...,NaN,NaN,qpoll_join_250326.xlsx,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 📊 6. Quickpoll 파일 통합

Quickpoll 파일들을 기본 DataFrame에 통합합니다.

In [14]:
# --- 1단계: 모든 파일에서 고유한 패널 ID(mb_sn) 전체 명단 만들기 ---
print("--- 1단계: 전체 패널 마스터 명단 생성 시작 ---")
all_panel_ids = set()

for file in all_files:
    try:
        # 💡 header=1을 가정하고, '고유번호' 또는 'mb_sn' 컬럼만 읽어옵니다.
        # Welcome 파일에 대한 예외 처리를 추가합니다.
        header_row = 0 if 'Welcome' in file else 1
        df_id = pd.read_excel(file, header=header_row)
        
        if '고유번호' in df_id.columns:
            all_panel_ids.update(df_id['고유번호'].dropna().unique())
        elif 'mb_sn' in df_id.columns:
            all_panel_ids.update(df_id['mb_sn'].dropna().unique())
            
    except Exception as e:
        print(f"  [경고] {os.path.basename(file)} 파일 처리 중 오류: {e}")

# 고유한 ID 목록으로 최종 기준이 될 base_df 생성
base_df = pd.DataFrame(list(all_panel_ids), columns=['mb_sn'])
print(f"--- 마스터 명단 생성 완료! 총 고유 패널 수: {len(base_df)}명 ---")

# --- 2단계: 마스터 명단에 파일별 정보 채워넣기 (Left Join) ---
print("--- 2단계: 파일별 정보 병합 시작 ---")
welcome_files = glob.glob(os.path.join(base_path, 'Welcome/*.xlsx'))
quickpoll_files = glob.glob(os.path.join(base_path, 'Quickpoll/*.xlsx'))

# 먼저 Welcome 파일(핵심 인구통계 정보)을 병합합니다.
for file in welcome_files:
    print(f"병합 중: {os.path.basename(file)}")
    temp_df = pd.read_excel(file) # Welcome은 header가 첫 줄에 있다고 가정
    base_df = pd.merge(base_df, temp_df, on='mb_sn', how='left')

# 다음으로 Quickpoll 파일을 병합합니다.
common_info_cols = ['구분', 'mb_sn', '성별', '나이', '지역', '설문일시']
for file in quickpoll_files:
    filename = os.path.basename(file).split('.')[0]
    print(f"병합 중: {filename}.xlsx")
    temp_df = pd.read_excel(file, header=1)
    
    if '고유번호' in temp_df.columns:
        temp_df.rename(columns={'고유번호': 'mb_sn'}, inplace=True)
    
    if 'mb_sn' in temp_df.columns:
        answer_cols = [col for col in temp_df.columns if col not in common_info_cols]
        rename_dict = {col: f"{col}_{filename}" for col in answer_cols}
        temp_df.rename(columns=rename_dict, inplace=True)
        unique_answer_cols = list(rename_dict.values())
        merge_target_df = temp_df[['mb_sn'] + unique_answer_cols]
        base_df = pd.merge(base_df, merge_target_df, on='mb_sn', how='left')
    
    # 메모리 정리
    del temp_df
    gc.collect()

print("🎉 최종 데이터 통합 완료!")
print(f"최종 데이터 크기: {base_df.shape[0]}개의 행, {base_df.shape[1]}개의 열")

--- 1단계: 전체 패널 마스터 명단 생성 시작 ---
--- 마스터 명단 생성 완료! 총 고유 패널 수: 36113명 ---
--- 2단계: 파일별 정보 병합 시작 ---
병합 중: Welcome_2nd.xlsx
병합 중: Welcome_1st.xlsx
병합 중: qpoll_join_250106.xlsx
병합 중: qpoll_join_250604.xlsx
병합 중: qpoll_join_250716.xlsx
병합 중: qpoll_join_250624.xlsx
병합 중: qpoll_join_250326.xlsx
병합 중: qpoll_join_250310.xlsx
병합 중: qpoll_join_250605.xlsx
병합 중: qpoll_join_250107.xlsx
병합 중: qpoll_join_250116.xlsx
병합 중: qpoll_join_250317.xlsx
병합 중: qpoll_join_250710.xlsx
병합 중: qpoll_join_250707.xlsx
병합 중: qpoll_join_250204.xlsx
병합 중: qpoll_join_250619.xlsx
병합 중: qpoll_join_250224.xlsx
병합 중: qpoll_join_250623.xlsx
병합 중: qpoll_join_250620.xlsx
병합 중: qpoll_join_250616.xlsx
병합 중: qpoll_join_250704.xlsx
병합 중: qpoll_join_250206.xlsx
병합 중: qpoll_join_250617.xlsx
병합 중: qpoll_join_250226.xlsx
병합 중: qpoll_join_250123.xlsx
병합 중: qpoll_join_250709.xlsx
병합 중: qpoll_join_250610.xlsx
병합 중: qpoll_join_250702.xlsx
병합 중: qpoll_join_250714.xlsx
병합 중: qpoll_join_250221.xlsx
병합 중: qpoll_join_250626.xlsx
병합 중: qpoll_joi

## 🔍 7. 중복 데이터 처리

데이터 품질을 개선하기 위해 중복된 `mb_sn`을 확인하고 제거합니다.

In [15]:
# 데이터 중복 확인 및 분석
print("=== 데이터 중복 분석 ===")
print(f"전체 행 수: {len(base_df)}")
print(f"고유한 mb_sn 개수: {base_df['mb_sn'].nunique()}")
print(f"중복된 mb_sn 개수: {len(base_df) - base_df['mb_sn'].nunique()}")

# 중복된 mb_sn 확인
duplicated_sn = base_df[base_df.duplicated(subset=['mb_sn'], keep=False)]
print(f"\n중복된 mb_sn이 있는 행 수: {len(duplicated_sn)}")

if len(duplicated_sn) > 0:
    print("\n중복된 mb_sn 예시:")
    print(duplicated_sn[['mb_sn']].head(10))
    
    # 중복 제거 (첫 번째 값만 유지)
    print("\n중복 제거 중...")
    base_df_clean = base_df.drop_duplicates(subset=['mb_sn'], keep='first')
    print(f"중복 제거 후 행 수: {len(base_df_clean)}")
    
    # 정리된 데이터 저장
    output_path_clean = '../data/processed/01_consolidated_data.csv'
    base_df_clean.to_csv(output_path_clean, index=False, encoding='utf-8-sig')
    print(f"중복 제거된 데이터가 '{output_path_clean}'에 저장되었습니다.")
    
    # 원본 데이터도 업데이트
    base_df = base_df_clean
    print(f"base_df가 중복 제거된 데이터로 업데이트되었습니다.")
else:
    print("중복된 데이터가 없습니다.")

=== 데이터 중복 분석 ===
전체 행 수: 36172
고유한 mb_sn 개수: 36113
중복된 mb_sn 개수: 59

중복된 mb_sn이 있는 행 수: 118

중복된 mb_sn 예시:
                 mb_sn
857   w292212429085909
858   w292212429085909
1372  w290910299159039
1373  w290910299159039
1378  w291820849282929
1379  w291820849282929
1476       ybchotest11
1477       ybchotest11
1504  w291631084171808
1505  w291631084171808

중복 제거 중...
중복 제거 후 행 수: 36113
중복 제거된 데이터가 '../data/processed/01_consolidated_data.csv'에 저장되었습니다.
base_df가 중복 제거된 데이터로 업데이트되었습니다.


## 💾 8. 최종 결과 저장

통합된 데이터를 최종적으로 저장하고 결과를 확인합니다.

In [16]:
# --- 3단계: 최종 결과 확인 및 저장 ---
print("\n--- 3단계: 최종 결과 확인 ---")
print(f"최종 데이터 크기: {base_df.shape[0]}개의 행, {base_df.shape[1]}개의 열")

# 데이터 정보 확인
print("\n--- 데이터 정보 ---")
base_df.info()

# 데이터 미리보기
print("\n--- 데이터 미리보기 ---")
display(base_df.head())

# 메모리 사용량 확인
final_memory = base_df.memory_usage(deep=True).sum() / 1024**2
print(f"\n최종 메모리 사용량: {final_memory:.1f}MB")

# 최종 결과물 저장
output_dir = '../data/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"폴더 생성: {output_dir}")

output_path = '../data/processed/01_consolidated_data.csv'
base_df.to_csv(output_path, index=False, encoding='utf-8-sig')
print(f"통합된 데이터가 '{output_path}' 경로에 저장되었습니다.")
print(f"저장된 파일 크기: {base_df.shape}")


--- 3단계: 최종 결과 확인 ---
최종 데이터 크기: 36113개의 행, 67개의 열

--- 데이터 정보 ---
<class 'pandas.core.frame.DataFrame'>
Index: 36113 entries, 0 to 36171
Data columns (total 67 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   mb_sn                  36113 non-null  object 
 1   Q1                     28744 non-null  float64
 2   Q2                     13616 non-null  float64
 3   Q3                     28744 non-null  float64
 4   Q4                     28744 non-null  float64
 5   Q5                     28742 non-null  object 
 6   Q5_1                   21728 non-null  object 
 7   Q6                     26606 non-null  float64
 8   Q7                     25507 non-null  float64
 9   Q8                     28744 non-null  object 
 10  Q9_1                   28742 non-null  object 
 11  Q9_2                   28662 non-null  object 
 12  Q10_x                  28744 non-null  float64
 13  Q11_1                  21420 non-null  obje

,mb_sn,Q1,Q2,Q3,Q4,Q5,Q5_1,Q6,Q7,Q8,...,문항1_qpoll_join_250714,문항1_qpoll_join_250221,문항1_qpoll_join_250626,문항1_qpoll_join_250723,문항1_qpoll_join_250627,문항1_qpoll_join_250328,문항1_qpoll_join_250703,문항1_qpoll_join_250611,문항1_qpoll_join_250304,문항2_qpoll_join_250304
0,w444788329818662,2.0,NaN,2.0,3.0,영양사,8,3.0,7.0,"1,2,3,4,5,6,,,,10,11,,,,,16,,,,,,,23,24,25,,,28",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,w244805554161210,1.0,NaN,5.0,4.0,2,8,4.0,7.0,"1,2,3,4,5,,7,,9,,,,,,15,,,,,20,,22,23,24,25,,,28",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,w223705011501017,2.0,1.0,3.0,1.0,1,19,3.0,4.0,"1,2,3,4,5,,7,,,,11,,,,,,,,19,20,,22,23,24,25,,,",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,w132840461644233,2.0,1.0,3.0,1.0,14,NaN,NaN,NaN,"1,2,3,4,5,6,7,8,9,10,11,,13,14,,,17,18,,20,,22...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,w195764112093178,2.0,NaN,2.0,3.0,1,8,9.0,11.0,"1,2,3,4,5,6,,8,,10,11,12,,14,15,,17,18,19,20,,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



최종 메모리 사용량: 56.3MB
통합된 데이터가 '../data/processed/01_consolidated_data.csv' 경로에 저장되었습니다.
저장된 파일 크기: (36113, 67)
